# Unit-test (kinda)

In [1]:
# Really quick-test
!python model.py
!python loss.py
!python loupe.py

In [17]:
# This takes a couple of minutes
!python didemo.py

/home/escorciav/install/bin/miniconda3/envs/scientific/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
simple test
Time loading data/raw/train_data.json:  114.0846495628357
33005
{'num_segments': 6, 'description': "a brown rat goes into someone's hand then onto a cage.", 'dl_link': 'https://www.flickr.com/video_download.gne?id=2408598493', 'times': [[2, 2], [2, 2], [2, 2], [2, 2]], 'video': '54322086@N00_2408598493_274c77d26a.avi', 'annotation_id': 2, 'language_input': ['a', 'brown', 'rat', 'goes', 'into', 'someone', 's', 'hand', 'then', 'onto', 'a', 'cage']}
0 (50, 300)
1 12
rgb (8194,)
flow (2050,)
rgb (8194,)
flow (2050,)
rgb (8194,)
flow (2050,)
0 (21, 50, 300)
1 21
rgb (21, 8194)
flow (21, 2050)


# TODO

- Add NetVLAD

- Experiment NetVLAD vs LSTM

# Bash

Test training loop
```bash
for i in {001..015}; do python train.py --epochs 2 --gpu-id 1 &> $i".log"; done
```

# Sandbox

Always stacking, It's better than scrolling :)

In [17]:
import torch

x = torch.tensor([[11, 12, 13], [21, 22, 23], [31, 32, 33]])
print(x)
n = x.size()[0]        
x1 = torch.diag(x)
print(x1.shape)
print(x1)
x1 = x1.expand(n, n)
print(x1.shape)
print(x1)
x1 = x1.contiguous().view(-1,1)
print(x1.shape)
print(x1)
x1 = torch.cat((x1, x1),0) 

x2 = x.view(-1, 1)
print(x2)
x3 = x.transpose(0, 1).contiguous().view(-1,1)
print(x3)

x2 = torch.cat((x2,x3),0)

y = (1 - torch.eye(n)).view(-1, 1)
print(y)

tensor([[ 11,  12,  13],
        [ 21,  22,  23],
        [ 31,  32,  33]])
torch.Size([3])
tensor([ 11,  22,  33])
torch.Size([3, 3])
tensor([[ 11,  22,  33],
        [ 11,  22,  33],
        [ 11,  22,  33]])
torch.Size([9, 1])
tensor([[ 11],
        [ 22],
        [ 33],
        [ 11],
        [ 22],
        [ 33],
        [ 11],
        [ 22],
        [ 33]])
tensor([[ 11],
        [ 12],
        [ 13],
        [ 21],
        [ 22],
        [ 23],
        [ 31],
        [ 32],
        [ 33]])
tensor([[ 11],
        [ 21],
        [ 31],
        [ 12],
        [ 22],
        [ 32],
        [ 13],
        [ 23],
        [ 33]])
tensor([[ 0.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 0.],
        [ 1.],
        [ 1.],
        [ 1.],
        [ 0.]])


In [12]:
from torch.utils.data import Dataset, DataLoader

class Simple(Dataset):
    def __init__(self):
        self.x = list(range(5))
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        return x * 2, {'x': x, 'y': x + 5}
    
data = Simple()
data[0]

loader = DataLoader(data)

for i in loader:
    print(i)

[tensor([ 0]), {'x': tensor([ 0]), 'y': tensor([ 5])}]
[tensor([ 2]), {'x': tensor([ 1]), 'y': tensor([ 6])}]
[tensor([ 4]), {'x': tensor([ 2]), 'y': tensor([ 7])}]
[tensor([ 6]), {'x': tensor([ 3]), 'y': tensor([ 8])}]
[tensor([ 8]), {'x': tensor([ 4]), 'y': tensor([ 9])}]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


class NetVLAD(nn.Module):
    
    def __init__(self, cluster_size, feature_size, add_batch_norm=True):
        super(NetVLAD, self).__init__()
        self.feature_size = feature_size
        self.cluster_size = cluster_size
        self.add_batch_norm = add_batch_norm
        self.out_dim = cluster_size * feature_size
        self.clusters = nn.Parameter(
            (1 / math.sqrt(feature_size)) * torch.randn(feature_size, cluster_size))
        self.clusters2 = nn.Parameter(
            (1 / math.sqrt(feature_size)) * th.randn(1, feature_size, cluster_size))
        if add_batch_norm:
            self.batch_norm = nn.BatchNorm1d(cluster_size)

    def forward(self, x):
        max_sample = x.shape[1]
        x = x.view(-1, self.feature_size)
        assignment = th.matmul(x, self.clusters)

        if self.add_batch_norm:
            assignment = self.batch_norm(assignment)

        assignment = F.softmax(assignment, dim=1)
        assignment = assignment.view(-1, max_sample, self.cluster_size)

        a_sum = th.sum(assignment, -2, keepdim=True)
        a = a_sum * self.clusters2
        assignment = assignment.transpose(1, 2)

        x = x.view(-1, max_sample, self.feature_size)
        vlad = th.matmul(assignment, x)
        vlad = vlad.transpose(1, 2)
        vlad = vlad - a

        # L2 intra norm
        vlad = F.normalize(vlad)
        
        # flattening + L2 norm
        vlad = vlad.view(-1, self.cluster_size * self.feature_size)
        vlad = F.normalize(vlad)

        return vlad

class NetRVLAD(nn.Module):
    
    def __init__(self, cluster_size, feature_size, add_batch_norm=True):
        super(NetRVLAD, self).__init__()
        self.feature_size = feature_size
        self.cluster_size = cluster_size
        self.add_batch_norm = add_batch_norm
        self.out_dim = cluster_size * feature_size
        self.clusters = nn.Parameter(
            (1 / math.sqrt(feature_size)) * th.randn(feature_size, cluster_size))
        if self.add_batch_norm:
            self.batch_norm = nn.BatchNorm1d(cluster_size)

    def forward(self,x):
        max_sample = x.shape[1]
        x = x.view(-1, self.feature_size)
        assignment = th.matmul(x, self.clusters)

        if self.add_batch_norm:
            assignment = self.batch_norm(assignment)

        assignment = F.softmax(assignment, dim=1)
        assignment = assignment.view(-1, max_sample, self.cluster_size)
        assignment = assignment.transpose(1, 2)

        x = x.view(-1, max_sample, self.feature_size)
        rvlad = th.matmul(assignment, x)
        rvlad = rvlad.transpose(-1, 1)

        # L2 intra norm
        rvlad = F.normalize(rvlad)
        
        # flattening + L2 norm
        rvlad = rvlad.view(-1, self.cluster_size * self.feature_size)
        rvlad = F.normalize(rvlad)

        return rvlad